# Test library

In [ ]:
import os
import sys

# Add the parent directory to the path to find modules
sys.path.insert(0, os.path.abspath('..'))
os.chdir('..')
import earthaccess
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Login using environment variables
auth = earthaccess.login(strategy="environment")

# Search for Rome area
results = earthaccess.search_data(
    short_name="SRTMGL1",
    version="003",
    bounding_box=(12.35, 41.8, 12.65, 42.0)  # min lon, min lat, max lon, max lat
)

# Download tiles
paths = earthaccess.download(results, "./srtm_tiles")
print(paths)


# Append variables to CVS for training

In [ ]:
from shroomradar.src.topography import enrich_csv

enrich_csv(
    "data/negative_samples_within_land_10k_with_coords.csv",
    "data/negative_samples_within_land_10k_with_coords_topography.csv",
    out_dir="dem_tiles",  # Files will be organized in subfolders: dem/, slope/, aspect/, geomorphons/
    download_tiles=False,  # 🚨 Set to True if you want to download new tiles
    variables=["dem", "slope", "aspect", "geomorphons"]  # Specify which variables to append
)



# Append to vector file

In [ ]:
from shroomradar.src.topography import enrich_geojson


enrich_geojson(
    "data/grid_tuscany_forest.geojson",
    "data/grid_tuscany_with_topography.geojson",
    out_dir="dem_tiles",
    download_tiles=True
)


# Upload to gcp

In [ ]:
from shroomradar.src.gcp import upload_dem_tiles_to_gcs, list_gcs_dem_tiles


BUCKET_NAME = "mushroom-radar"

# Upload DEM tiles to GCS
print("Uploading DEM tiles to Google Cloud Storage...")
uploaded = upload_dem_tiles_to_gcs(
    local_dir="..//dem_tiles//slope",
    bucket_name=BUCKET_NAME,
    gcs_prefix="topography/slope/"
)

# List files in bucket
print("\nListing files in bucket...")
list_gcs_dem_tiles(bucket_name=BUCKET_NAME, gcs_prefix="topography/slope")

# Download DEM tiles from GCS (example)
# print("\nDownloading DEM tiles from Google Cloud Storage...")
# downloaded = download_dem_tiles_from_gcs(
#     bucket_name=BUCKET_NAME,
#     gcs_prefix="dem_tiles/",
#     local_dir="dem_tiles_downloaded"
# )


Uploading DEM tiles to Google Cloud Storage...
Found 4284 .tif files to upload


Uploading to GCS:   0%|          | 1/4284 [00:02<2:35:11,  2.17s/it]

✅ Uploaded: topography/slope/N00E011_slope.tif


Uploading to GCS:   0%|          | 2/4284 [00:03<1:55:36,  1.62s/it]

✅ Uploaded: topography/slope/N00E013_slope.tif


...

✅ Uploaded: topography/slope/N17W010_slope.tif


Uploading to GCS:  11%|█         | 476/4284 [11:57<2:04:17,  1.96s/it]

✅ Uploaded: topography/slope/N17W014_slope.tif


Uploading to GCS:  11%|█         | 477/4284 [11:59<1:55:52,  1.83s/it]

✅ Uploaded: topography/slope/N17W089_slope.tif
